# __Fetch aoe2recs.com match details__

### __Load libraries & source functions__

In [ ]:
# Libraries
from pymongo import MongoClient

from tqdm import tqdm
from itertools import cycle
from bs4 import BeautifulSoup
import pandas as pd, numpy as np
import os, math, random, time, warnings, json

import requests, mechanize, http.cookiejar

from joblib import Parallel, delayed

pd.set_option('display.max_rows', 50)oc 
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Source
%run aoe2rec_api_modules.ipynb

### __Fetch match details__

In [ ]:
fetch_multiple_matches_iterate_aoe2recs(db_conn = setup_mongo_conn(), collect = 300000, batch_size = 50)